In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# Data Preprocessing

In [ ]:
data = pd.read_csv('data.csv')


In [ ]:
data.head()

In [ ]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

In [ ]:
print(X)
print(y)

## Data Encoding

In [ ]:
X_encoded = pd.get_dummies(X, columns=['Gender'])

In [ ]:
print(X_encoded)

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
print(y_encoded)

## Standard Feature Scaling

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_encoded)

In [ ]:
print(X_scaled)

## ANOVA F-Value Feature Selection

In [ ]:
k = 5
selector = SelectKBest(score_func=f_classif, k=k)
X_selected = selector.fit_transform(X_scaled, y_encoded)

In [ ]:
selected_features = selector.get_support()
feature_names = X_encoded.columns
selected_feature_names = feature_names[selected_features]

print("Selected Features:")
for feature in selected_feature_names:
    print(feature)

print(X_selected)

## Training and Test Data Allocation

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_selected, y_encoded, test_size=0.2, random_state=42)


# Model Training and Prediction

## Hyperparameter Optimization

In [ ]:
# param_grid = [
#     {'activation': ['relu', 'tanh', 'logistic', 'identity'],
#      'hidden_layer_sizes': [(100, 50, 25), (50, 25), (25)],
#     'alpha': [0.1, 0.01, 0.001, 0.0001],
#     'solver': ['lbfgs', 'sgd', 'adam'],
#     'learning_rate': ['constant', 'adaptive', 'invscaling']
#      }
# ]

# optimal_params = GridSearchCV(
#     MLPClassifier(max_iter=1000),
#     param_grid,
#     cv = 5,
#     scoring = 'accuracy',
#     verbose = 0
# )

In [ ]:
# optimal_params.fit(X_train, y_train)
# print(optimal_params.best_params_)

## Training

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(100, 50, 25), activation='tanh', alpha=0.1, solver='lbfgs', learning_rate='constant')
mlp.fit(X_train, y_train)

## Prediction

In [ ]:
y_pred = mlp.predict(X_test)

# Results

## Decoding

In [ ]:
y_pred_decoded = label_encoder.inverse_transform(y_pred)
y_test_decoded = label_encoder.inverse_transform(y_test)

## Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test_decoded, y_pred_decoded)
cm_fig = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = ['ND', 'PD', 'D'])
cm_fig.plot()
plt.show()

## Classification Report

In [ ]:
print(classification_report(y_test_decoded, y_pred_decoded))